In [2]:
import re
import io
import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter

In [5]:
# import parser from tika
from tika import parser
 
# get a sample pdf
parsedPDF = parser.from_file("bax088.pdf")
 
# print parsed PDF data
#print (parsedPDF)
#print (parsedPDF.viewkeys())
#print (parsedPDF["metadata"])
#print (parsedPDF["content"])

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

abstract = find_between(parsedPDF["content"], "Abstract", "Introduction" )
intro = find_between(parsedPDF["content"], "Introduction", "Background" )
background = find_between(parsedPDF["content"], "Background", "Materials" )
method = find_between(parsedPDF["content"], "Materials and methods", "Results" )


print("Background :", background)

#print (find_between(parsedPDF["content"], "Abstract", "Introduction" ))
#print (find_between_r( parsedPDF["content"], "Abstract", "Introduction"))

#p = re.compile(r'?:ABSTRACT\s*\n+|Abstract\s*\n+)(.*?)((?:[A-Z]+|(?:\n(?:[A-Z]+|(?:[A-Z][a-z]+\s*)+)\n+)')

#re.findall(p, parsedPDF["content"])

2019-03-03 04:38:41,575 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2019-03-03 04:38:57,631 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2019-03-03 04:38:58,169 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2019-03-03 04:39:03,177 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Background : 

In recent years, several annotation formats have been

advanced to store and distribute biomedical information

extraction outcomes. Commonly called annotations, they

are generated following a specific structure or format de-

pendent on the extraction system, and integration with ex-

ternal databases and systems is challenging. IeXML (16)

was one of the first XML-based implementations to define

an exchange format considering annotations and enrich-

ment of text. More recently, the BioC (17) has emerged as

a community-supported format for encoding and sharing

textual annotations. This simplified approach streamlines

data reuse and sharing methods, achieving interoperability

for the different text processing tasks by defining con-

nectors to read and write XML annotations. Although

they are created to enable interoperability and reusability

between text-mined systems, these data structures are not

designed to support data exploration and sustainability.

To a

In [24]:
###tokenizing abstract#####
### store the last sentence of abstract in abc1.txt file

import nltk
from nltk.tokenize import sent_tokenize
import nltk.data

abstract_sent =sent_tokenize(abstract)
#print(abstract_sent)

for i,sent in enumerate(abstract_sent):
    print(sent)
    
with open('abc1.txt', 'w') as f:
    print(sent, file=f)
    
    
#f = open('abc1.txt','w')
#print(abstract_sent, file=f)



Computational annotation of textual information has taken on an important role in knowledge

extraction from the biomedical literature, since most of the relevant information from scien-

tific findings is still maintained in text format.
In this endeavour, annotation tools can assist in

the identification of biomedical concepts and their relationships, providing faster reading and

curation processes, with reduced costs.
However, the separate usage of distinct annotation

systems results in highly heterogeneous data, as it is difficult to efficiently combine and ex-

change this valuable asset.
Moreover, despite the existence of several annotation formats,

there is no unified way to integrate miscellaneous annotation outcomes into a reusable, shar-

able and searchable structure.
Taking up this challenge, we present a modular architecture

for textual information integration using semantic web features and services.
The solution

described allows the migration of curation data int

In [25]:
### Summarizing the text ####
### here I am summarizing introduction part - intro"
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import nltk




stemmer = SnowballStemmer("english")
stopWords = set(stopwords.words("english"))
words = word_tokenize(intro)

freqTable = dict()
for word in words:
    word = word.lower()
    if word in stopWords:
        continue

    word = stemmer.stem(word)

    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1

sentences = sent_tokenize(intro)
sentenceValue = dict()

for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq



sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]

# Average value of a sentence from original text
average = int(sumValues / len(sentenceValue))


summary = ''
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
        summary += " " + sentence

#print(summary)

with open('abc2.txt', 'a') as f1:
    print(summary, file=f1)

In [ ]:
## now we have last sentence of abstarct in abc1 and Summarized text from introduction in abc2
## now let us see the similarity using cosine similarity

In [26]:
from nltk.corpus import stopwords
stopwords_en = stopwords.words("english")
def preprocessing(raw):
    wordlist = nltk.word_tokenize(raw)
    text = [w.lower() for w in wordlist if w not in stopwords_en]
    return text
                               
f1 = open('abc1.txt','r',encoding ='utf-8')
text1 = preprocessing(f1.read())

                               
f2 = open('abc2.txt','r',encoding ='utf-8')
text2 = preprocessing(f2.read())

#f3 = open('abc3.txt','r',encoding ='utf-8')
#text3 = preprocessing(f3.read())

In [28]:
## Frequency count method
from nltk.probability import FreqDist
word_set = set(text1).union(set(text2))

freqd_text1 = FreqDist(text1)
text1_count_dict = dict.fromkeys(word_set,0)
for word in text1:
    text1_count_dict[word]= freqd_text1[word]

freqd_text2 = FreqDist(text2)
text2_count_dict = dict.fromkeys(word_set,0)
for word in text2:
    text2_count_dict[word]= freqd_text2[word]

#text1_count_dict
v1 = list(text1_count_dict.values())
v2 = list(text2_count_dict.values())

similarity = 1 - nltk.cluster.cosine_distance(v1,v2)
print("Similarity Index : {:4.2f} % ".format(similarity*100))

Similarity Index : 36.72 % 


In [31]:
#TF calculations
freqd_text1 = FreqDist(text1)
text1_length = len(text1)
text1_tf_dict = dict.fromkeys(word_set,0)
for word in text1:
    text1_tf_dict[word]= freqd_text1[word]/text1_length

freqd_text2 = FreqDist(text2)
text2_length = len(text2)
text2_tf_dict = dict.fromkeys(word_set,0)
for word in text2:
    text2_tf_dict[word]= freqd_text2[word]/text2_length

In [32]:
## IDF calculations

text12_idf_dict = dict.fromkeys(word_set,0)
text12_length = 2
for word in text12_idf_dict.keys():
    if word in text1:
        text12_idf_dict[word] +=1
    if word in text2:
        text12_idf_dict[word] +=1
        
import math
for word,val in text12_idf_dict.items():
    text12_idf_dict[word] = 1 + math.log(text12_length/(float(val)))

In [33]:
## TDIF of a word

text1_tfidf_dict = dict.fromkeys(word_set,0)
for word in text1:
    text1_tfidf_dict[word] = (text1_tf_dict[word])*(text12_idf_dict[word])

text2_tfidf_dict = dict.fromkeys(word_set,0)
for word in text2:
    text2_tfidf_dict[word] = (text2_tf_dict[word])*(text12_idf_dict[word])

In [34]:
## compute cosine similarity

v1 = list(text1_tfidf_dict.values())
v2 = list(text2_tfidf_dict.values())
similarity = 1 - nltk.cluster.cosine_distance(v1,v2)
print("Similarity Index : {:4.2f} % ".format(similarity*100))


Similarity Index : 18.14 % 


In [36]:
## doc2vec
import gensim
from gensim.models.doc2vec import TaggedDocument

taggeddocs=[]

doc1 = TaggedDocument(words=text1,tags =[u'Abstract'])
taggeddocs.append(doc1)
#doc2 = TaggedDocument(words=text2,tags =[u'Intro'])
#taggeddocs.append(doc2)


doc3 = TaggedDocument(words=text2,tags =[u'Background'])
taggeddocs.append(doc3)
## build the model

model = gensim.models.Doc2Vec(taggeddocs,dm=0,alpha = 0.025, size=10, min_alpha=0.025, min_count=0)

#training


for epoch in range(80):
    if epoch % 10 == 0:
        print("Training epoch %s" % epoch)
    model.train(taggeddocs,total_examples=model.corpus_count,epochs=model.epochs)
    model.alpha -=0.002
    model.min_alpha = model.alpha

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:535: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Training epoch 0
Training epoch 10
Training epoch 20
Training epoch 30
Training epoch 40
Training epoch 50
Training epoch 60
Training epoch 70


In [38]:
similarity = model.n_similarity(text1,text2)
print("Similarity Index : {:4.2f} % ".format(similarity*100)) 


Similarity Index : 63.49 % 


/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
